In [200]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [201]:
# Set up Splinter - set the executable path and initialize a browser:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been 

In [202]:
# This code tells Splinter which site we want to visit by assigning the link to a URL
character_name = 'zarude'
url = f'https://pokemondb.net/pokedex/{character_name}'
browser.visit(url)

In [203]:
# Parse the HTML: use BeautifulSoup to parse the HTML
html = browser.html
html_soup = soup(html, 'html.parser')

In [204]:
get_table_data = html_soup.find('div', class_='sv-tabs-panel-list')

## Getting Pokedex Data

In [205]:
# GET Pokédex data / Vitals:
pokedex_data_table = get_table_data.find(class_='vitals-table')

In [206]:
# Get Table Data
table_body = pokedex_data_table.find('tbody')
rows = table_body.find_all('tr')

table_head = []
table_data = []

for row in rows:
    th = row.find_all('th')
    td = row.find_all('td')
    th = [x.text.strip() for x in th]
    td = [x.text.strip() for x in td]  
    table_head.append(th)
    table_data.append(td)

# Create Pokédex data / Vitals dataframe::
pokedex_head_df = pd.DataFrame(table_head)
pokedex_data_df = pd.DataFrame(table_data)
pokedex_table_df = pd.concat([pokedex_head_df, pokedex_data_df], axis=1)


## Getting Training Data:

In [207]:
training = get_table_data.find(class_='grid-col span-md-12 span-lg-4')
training_data = training.find_all(class_='vitals-table')[0]

# GET Training Data:
table_body = training_data.find('tbody')
rows = table_body.find_all('tr')

table_head = []
table_data = []

for row in rows:
    th=row.find_all('th')
    td=row.find_all('td')
    th=[x.text.strip() for x in th]
    td=[x.text.strip() for x in td]  
    table_head.append(th)
    table_data.append(td)

# Create Pokédex data / Vitals dataframe::
training_head = pd.DataFrame(table_head)
training_data = pd.DataFrame(table_data)
training_df = pd.concat([training_head, training_data], axis=1)
#training_df.head(10)

## Getting Breeding Data:

In [208]:
breeding = training.find_all(class_='vitals-table')[1]

# GET Training Data:
table_body = breeding.find('tbody')
rows = table_body.find_all('tr')

table_head = []
table_data = []

for row in rows:
    th=row.find_all('th')
    td=row.find_all('td')
    th=[x.text.strip() for x in th]
    td=[x.text.strip() for x in td]  
    table_head.append(th)
    table_data.append(td)

# Create Pokédex data / Vitals dataframe::
breeding_head = pd.DataFrame(table_head)
breeding_data = pd.DataFrame(table_data)
breeding_df = pd.concat([breeding_head, breeding_data], axis=1)
#breeding_df.head(10)



## Getting Base Stats Data

In [209]:
# GET Base Stats
base_stats = get_table_data.find(class_='grid-col span-md-12 span-lg-8')

# GET Base Stats Data:
table_body = base_stats.find('tbody')
rows = table_body.find_all('tr')

table_head = []
table_data = []

for row in rows:
    th=row.find_all('th')
    td=row.find_all('td')
    th=[x.text.strip() for x in th]
    td=[x.text.strip() for x in td]  
    table_head.append(th)
    table_data.append(td)

# Create Pokédex data / Vitals dataframe::
stats_head = pd.DataFrame(table_head)
stats_data = pd.DataFrame(table_data)
stats_df = pd.concat([stats_head, stats_data], axis=1)
#stats_df.head(10)

## Displaying Collected Data:

In [211]:
pokedex_table_df

,Type,Value
0,National №,893
1,Type,Dark Grass
2,Species,Rogue Monkey Pokémon
3,Height,1.8 m (5′11″)
4,Weight,70.0 kg (154.3 lbs)
5,Abilities,1. Leaf Guard
6,Local №,211 (The Isle of Armor)


In [251]:
training_df.head(10)

,0,0
0,EV yield,3 Attack
1,Catch rate,"3 (0.4% with PokéBall, full HP)"
2,Base Friendship,0 (lower than normal)
3,Base Exp.,300
4,Growth Rate,Slow


In [213]:
breeding_df.head(10)

,0,0
0,Egg Groups,Undiscovered
1,Gender,Genderless
2,Egg cycles,"120 (30,584–30,840 steps)"


In [214]:
stats_df.head(10)

,0,0,1,2,3
0,HP,105,,320,414
1,Attack,120,,220,372
2,Defense,105,,193,339
3,Sp. Atk,70,,130,262
4,Sp. Def,95,,175,317
5,Speed,105,,193,339


In [239]:
Status_value = stats_df.iloc[: , [0, 1]].copy().reset_index(drop = True)
Status_value = Status_value.transpose()
Status_value

,0,1,2,3,4,5
0,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,105,120,105,70,95,105


In [240]:
Status_value_min = stats_df.iloc[: , [0, 3]].copy().reset_index(drop = True)
Status_value_min = Status_value_min.add_prefix('min_')
Status_value_min = Status_value_min.transpose()
Status_value_min

,0,1,2,3,4,5
min_0,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
min_2,320,220,193,130,175,193


In [234]:
Status_value_max = stats_df.iloc[: , [0, 4]].copy().reset_index(drop = True)
Status_value_max = Status_value_max.transpose()
Status_value_max

,0,1,2,3,4,5
0,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
3,414,372,339,262,317,339
